# Suggestion Writer in Python

This code is main edited by Enze Wang

# Attention

Please put four .csv file in data folder and put this code in main folder instead of code folder.

This code want to write suggesion for business from the analysis above.  



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import itertools
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from nltk import FreqDist
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
from wordcloud import WordCloud
from nltk import sent_tokenize, word_tokenize, PorterStemmer, pos_tag 
from nltk.corpus import stopwords
from nltk.text import TextCollection
from nltk.corpus import wordnet
import nltk.tokenize as nt
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /Users/enze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/enze/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/enze/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/enze/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/enze/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
Business_results=pd.read_csv("data/Results_Analysis_Business_Class.csv")
Noun_results=pd.read_csv("data/Results_Analysis_Noun_Class.csv")
Review_results=pd.read_csv("data/Results_Analysis_Noun_Attitude.csv")
pizza_review=pd.read_csv("data/pizza_review.csv")
pizza_business=pd.read_csv("data/pizza_business.csv")
pizza_business_clean_review=pd.read_csv("data/pizza_business_clean_review.csv")
Word_Review_Matrix=pd.read_csv("data/Word_Review_Matrix.csv")

# Business Suggestion

In [3]:
Business_results

,Class,Category,Mean
0,Alcohol,beer_and_wine,3.573333
1,Alcohol,full_bar,3.555385
2,Alcohol,None,3.387755
3,BikeParking,FALSE,3.366987
4,BikeParking,TRUE,3.556686
5,GoodForKids,FALSE,3.186335
6,GoodForKids,TRUE,3.420018
7,NoiseLevel,average,3.517665
8,NoiseLevel,loud,3.296053
9,NoiseLevel,quiet,3.430952


Suggestion for business is simple. For example, if one business offer Alcohol, we can suggest them to keep. If not, suggest them to offer alcohol as possible.  

Therefore in our suggestion here, we use 1 as the business rating is high option and 0 as the low option. The detailed suggestion will raise by Shiny app code. 

For np.nan in the value, we default give low option rating suggestion. 

Some special category, such as Delivery, we will mention and give warning to those given delivery to raise their delivery quality and service instead of cancel delivery.

In [4]:
pizza_business["Alcohol"]

0                None
1                 NaN
2                None
3       beer_and_wine
4       beer_and_wine
            ...      
1555             None
1556             None
1557             None
1558             None
1559         full_bar
Name: Alcohol, Length: 1560, dtype: object

In [5]:
set(pizza_business["Alcohol"])

{'None', 'beer_and_wine', 'full_bar', nan}

In [6]:
Alcohol_Suggestion=pizza_business["Alcohol"]
Alcohol_Suggestion=Alcohol_Suggestion.replace(['beer_and_wine','full_bar'],"1")
Alcohol_Suggestion=Alcohol_Suggestion.replace(['None',np.nan],"0")

In [7]:
Alcohol_Suggestion

0       0
1       0
2       0
3       1
4       1
       ..
1555    0
1556    0
1557    0
1558    0
1559    1
Name: Alcohol, Length: 1560, dtype: object

In [8]:
pizza_business["BikeParking"]

0        True
1         NaN
2        True
3         NaN
4        True
        ...  
1555      NaN
1556    False
1557    False
1558     True
1559     True
Name: BikeParking, Length: 1560, dtype: object

In [9]:
set(pizza_business["BikeParking"])

{nan, False, True}

In [10]:
BikeParking_Suggestion=pizza_business["BikeParking"]
BikeParking_Suggestion=BikeParking_Suggestion.replace([True],"1")
BikeParking_Suggestion=BikeParking_Suggestion.replace([False,np.nan],"0")
BikeParking_Suggestion

0       1
1       0
2       1
3       0
4       1
       ..
1555    0
1556    0
1557    0
1558    1
1559    1
Name: BikeParking, Length: 1560, dtype: object

In [11]:
pizza_business["GoodForKids"]

0       True
1       True
2       True
3       True
4       True
        ... 
1555    True
1556    True
1557    True
1558    True
1559    True
Name: GoodForKids, Length: 1560, dtype: object

In [12]:
set(pizza_business["GoodForKids"])

{nan, False, True}

In [13]:
GoodForKids_Suggestion=pizza_business["GoodForKids"]
GoodForKids_Suggestion=GoodForKids_Suggestion.replace([True],"1")
GoodForKids_Suggestion=GoodForKids_Suggestion.replace([False,np.nan],"0")
GoodForKids_Suggestion

0       1
1       1
2       1
3       1
4       1
       ..
1555    1
1556    1
1557    1
1558    1
1559    1
Name: GoodForKids, Length: 1560, dtype: object

In [14]:
set(GoodForKids_Suggestion)

{'0', '1'}

In [15]:
pizza_business["NoiseLevel"]

0           quiet
1           quiet
2           quiet
3           quiet
4         average
          ...    
1555          NaN
1556      average
1557          NaN
1558      average
1559    very_loud
Name: NoiseLevel, Length: 1560, dtype: object

In [16]:
set(pizza_business["NoiseLevel"])

{'average', 'loud', nan, 'quiet', 'very_loud'}

In [17]:
NoiseLevel_Suggestion=pizza_business["NoiseLevel"]
NoiseLevel_Suggestion=NoiseLevel_Suggestion.replace(['average','quiet'],"1")
NoiseLevel_Suggestion=NoiseLevel_Suggestion.replace(['very_loud',np.nan,'loud'],"0")
NoiseLevel_Suggestion

0       1
1       1
2       1
3       1
4       1
       ..
1555    0
1556    1
1557    0
1558    1
1559    0
Name: NoiseLevel, Length: 1560, dtype: object

In [18]:
set(NoiseLevel_Suggestion)

{'0', '1'}

In [19]:
pizza_business["RestaurantsGoodForGroups"]

0        True
1       False
2        True
3        True
4        True
        ...  
1555     True
1556     True
1557     True
1558     True
1559     True
Name: RestaurantsGoodForGroups, Length: 1560, dtype: object

In [20]:
set(pizza_business["RestaurantsGoodForGroups"])

{False, True, nan}

In [21]:
RestaurantsGoodForGroups_Suggestion=pizza_business["RestaurantsGoodForGroups"]
RestaurantsGoodForGroups_Suggestion=RestaurantsGoodForGroups_Suggestion.replace([True],"1")
RestaurantsGoodForGroups_Suggestion=RestaurantsGoodForGroups_Suggestion.replace([False,np.nan],"0")
RestaurantsGoodForGroups_Suggestion

0       1
1       0
2       1
3       1
4       1
       ..
1555    1
1556    1
1557    1
1558    1
1559    1
Name: RestaurantsGoodForGroups, Length: 1560, dtype: object

In [22]:
set(RestaurantsGoodForGroups_Suggestion)

{'0', '1'}

In [23]:
pizza_business["RestaurantsReservations"]

0        True
1       False
2       False
3       False
4       False
        ...  
1555    False
1556    False
1557    False
1558    False
1559    False
Name: RestaurantsReservations, Length: 1560, dtype: object

In [24]:
set(pizza_business["RestaurantsReservations"])

{False, True, nan}

In [25]:
RestaurantsReservations_Suggestion=pizza_business["RestaurantsReservations"]
RestaurantsReservations_Suggestion=RestaurantsReservations_Suggestion.replace([True],"1")
RestaurantsReservations_Suggestion=RestaurantsReservations_Suggestion.replace([False,np.nan],"0")
RestaurantsReservations_Suggestion

0       1
1       0
2       0
3       0
4       0
       ..
1555    0
1556    0
1557    0
1558    0
1559    0
Name: RestaurantsReservations, Length: 1560, dtype: object

In [26]:
set(RestaurantsReservations_Suggestion)

{'0', '1'}

In [27]:
pizza_business["WiFi"]

0         no
1         no
2         no
3         no
4       free
        ... 
1555      no
1556      no
1557    free
1558      no
1559      no
Name: WiFi, Length: 1560, dtype: object

In [28]:
set(pizza_business["WiFi"])

{'free', nan, 'no', 'paid'}

In [29]:
WiFi_Suggestion=pizza_business["WiFi"]
WiFi_Suggestion=WiFi_Suggestion.replace(['free'],"1")
WiFi_Suggestion=WiFi_Suggestion.replace(['no','paid',np.nan],"0")
WiFi_Suggestion

0       0
1       0
2       0
3       0
4       1
       ..
1555    0
1556    0
1557    1
1558    0
1559    0
Name: WiFi, Length: 1560, dtype: object

In [30]:
set(WiFi_Suggestion)

{'0', '1'}

In [31]:
pizza_business["RestaurantsDelivery"]

0        True
1        True
2        True
3        True
4        True
        ...  
1555      NaN
1556     True
1557    False
1558     True
1559     True
Name: RestaurantsDelivery, Length: 1560, dtype: object

In [32]:
set(pizza_business["RestaurantsDelivery"])

{False, True, nan}

In [33]:
RestaurantsDelivery_Suggestion=pizza_business["RestaurantsDelivery"]
RestaurantsDelivery_Suggestion=RestaurantsDelivery_Suggestion.replace([False],"1")
RestaurantsDelivery_Suggestion=RestaurantsDelivery_Suggestion.replace([True,np.nan],"0")
RestaurantsDelivery_Suggestion

0       0
1       0
2       0
3       0
4       0
       ..
1555    0
1556    0
1557    1
1558    0
1559    0
Name: RestaurantsDelivery, Length: 1560, dtype: object

In [34]:
set(RestaurantsDelivery_Suggestion)

{'0', '1'}

In [35]:
pizza_business["BusinessParking"]

0          NaN
1          NaN
2          NaN
3          lot
4          lot
         ...  
1555       NaN
1556       NaN
1557       NaN
1558    street
1559       NaN
Name: BusinessParking, Length: 1560, dtype: object

In [36]:
set(pizza_business["BusinessParking"])

{'garage',
 'lot',
 'lot street',
 'lot street garage',
 nan,
 'street',
 'street garage',
 'valet',
 'valet lot',
 'valet street',
 'valet street garage',
 'validated',
 'validated lot',
 'validated street garage',
 'validated valet',
 'validated valet street garage'}

In [37]:
BusinessParking_Suggestion=pizza_business["BusinessParking"]
BusinessParking_Suggestion=BusinessParking_Suggestion.replace(['garage','lot',
 'lot street',
 'lot street garage',
 'street',
 'street garage',
 'valet',
 'valet lot',
 'valet street',
 'valet street garage',
 'validated',
 'validated lot',
 'validated street garage',
 'validated valet',
 'validated valet street garage'],"1")
BusinessParking_Suggestion=BusinessParking_Suggestion.replace([np.nan],"0")
BusinessParking_Suggestion

0       0
1       0
2       0
3       1
4       1
       ..
1555    0
1556    0
1557    0
1558    1
1559    0
Name: BusinessParking, Length: 1560, dtype: object

In [38]:
set(BusinessParking_Suggestion)

{'0', '1'}

In [39]:
Business_Suggestion=pd.DataFrame([pizza_business["business_id"],
                                  Alcohol_Suggestion,
BikeParking_Suggestion,
GoodForKids_Suggestion,
NoiseLevel_Suggestion,
RestaurantsGoodForGroups_Suggestion,
RestaurantsReservations_Suggestion,
WiFi_Suggestion,
RestaurantsDelivery_Suggestion,
BusinessParking_Suggestion])
Business_Suggestion=Business_Suggestion.transpose()
Business_Suggestion

,business_id,Alcohol,BikeParking,GoodForKids,NoiseLevel,RestaurantsGoodForGroups,RestaurantsReservations,WiFi,RestaurantsDelivery,BusinessParking
0,ZkzutF0P_u0C0yTulwaHkA,0,1,1,1,1,1,0,0,0
1,39lLJK_rrYY2NYomSsQdUA,0,0,1,1,0,0,0,0,0
2,0y6alZmSLnPzmG5_kP5Quw,0,1,1,1,1,0,0,0,0
3,_Kp1IPTi17wBywYOd30raA,1,0,1,1,1,0,0,0,1
4,crBw_Hq3gCkppirgjnIl4A,1,1,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,0,0,1,0,1,0,0,0,0
1556,ZZGs9idRiKRHue2RyXw9uA,0,0,1,1,1,0,0,0,0
1557,v8GgrqLTs_NZZl2tkyvOgA,0,0,1,0,1,0,1,1,0
1558,qWcaGio9JsqrvgmQiIRtIQ,0,1,1,1,1,0,0,0,1


Check, the business_id 0y6alZmSLnPzmG5_kP5Quw

In [40]:
Business_Suggestion.iloc[2]

business_id                 0y6alZmSLnPzmG5_kP5Quw
Alcohol                                          0
BikeParking                                      1
GoodForKids                                      1
NoiseLevel                                       1
RestaurantsGoodForGroups                         1
RestaurantsReservations                          0
WiFi                                             0
RestaurantsDelivery                              0
BusinessParking                                  0
Name: 2, dtype: object

In pizza_business:

In [41]:
pizza_business.iloc[2]

Unnamed: 0                                              2
business_id                        0y6alZmSLnPzmG5_kP5Quw
name                                          J J's Pizza
address                                   20542 Lorain Rd
city                                            Cleveland
latitude                                          41.4483
longitude                                        -81.8476
stars                                                 4.5
review_count                                           21
categories                    Pizza, Italian, Restaurants
Alcohol                                              None
Ambience                                              NaN
BikeParking                                          True
BusinessAcceptsCreditCards                           True
Caters                                               True
GoodForKids                                          True
HasTV                                                True
NoiseLevel    

No problem

# Noun suggestion

The noun suggestion suggest business to offer or add XXX service to their business, something like food or service. We will raise suggestion like:

Review contains XXX, XXX and XXX individually always have high review star rating compared with that without. Think about add word related service or food to increase your star rating. 

or

Review contains XXX, XXX and XXX individually always have low review star rating compared with that without. Think about remove or fix word related service or food to increase your star rating. 

The Noun suggestion is the most difficult part in our suggestion, because:

1. Some review contains "cheese" for example may something like "I want you offer cheese pizza". Therefore it is strange to provide suggestion like "Keep cheese in your food list" if this business review does not contain "cheese" but it does offer cheese.

Our suggestion will be divided into four part. Add, Keep, Fix, Avoid. 

Add means that you do not have positive noun and we suggest you add. 

Keep means that you do have positive noun and we suggest you keep. 

Fix means that you do have negative noun and we suggest you fix. 

Avoid means that you do not have negative noun and we suggest you avoid. 

In [42]:
Noun_results=Noun_results.rename(columns={"Unnamed: 0": "Word"})
Noun_results

,Word,Linear_Coefficients,Linear_STD_Error,Linear_T_Value,Linear_P_Value
0,beer,0.062833,0.010396,6.044049,1.510856e-09
1,pepperoni,0.032375,0.015827,2.045537,4.080582e-02
2,meatball,0.075464,0.014579,5.176072,2.272592e-07
3,wine,0.106070,0.015682,6.763890,1.354361e-11
4,sausage,0.062473,0.019530,3.198854,1.380396e-03
5,mushroom,-0.062621,0.020695,-3.025896,2.479914e-03
6,pepper,0.043834,0.021286,2.059312,3.946823e-02
7,dessert,0.126776,0.023900,5.304513,1.133325e-07
8,bacon,0.088723,0.025136,3.529779,4.161864e-04
9,mozzarella,0.090639,0.029072,3.117723,1.823314e-03


In [43]:
Positive_Noun=Noun_results[Noun_results["Linear_Coefficients"]>0]["Word"].tolist()
Negative_Noun=Noun_results[Noun_results["Linear_Coefficients"]<0]["Word"].tolist()

In [44]:
pizza_business_clean_review

,Unnamed: 0,review_id,clean_review
0,0,hfXVxrlD6fr1kXfz6sR13Q,parking tough get get go much also get little ...
1,1,EDQpLLFiAxeSxWI5ipwzBQ,server extensive knowledge wine various menu s...
2,2,RWmT90rP7wYo57dG1cyvww,fast forward every trip nyc pitt includes visi...
3,3,0jduztItUhOh3FzKGjMxZQ,far favorite cleveland/akron restaurant great ...
4,4,sxfmzhMvZ8LlI1x4fhNjKg,food good usually get italian sub pizza would ...
...,...,...,...
67164,67164,gcNRaJMDlSVs8YlqwpsRuw,originally side ohio always really enjoyed jet...
67165,67165,yFxDOdiDb8HPaK1io81kiw,yes yes yes loved going place shadyside ecstat...
67166,67166,Z_xmoLgIMB4As90t0a5eBQ,went lunch meeting amidst typical suburban str...
67167,67167,CzYRFCnkbqOVb3WIMMrbfw,wife eaten la brioche least dozen time food ne...


In [45]:
pizza_business_review=pd.merge(pizza_review, pizza_business_clean_review, how='left', on='review_id')

In [46]:
pizza_business_review

,Unnamed: 0_x,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Unnamed: 0_y,clean_review
0,0,hfXVxrlD6fr1kXfz6sR13Q,7nIzgNF7YzI-UAyYr3y9hw,J9f-9Prw2YVM-fiZqv2fmQ,2,0,1,1,Parking is tough to get here so I don't get to...,2013-09-04 04:35:27,0,parking tough get get go much also get little ...
1,1,EDQpLLFiAxeSxWI5ipwzBQ,SkueY7hVsyXmYItELoodGw,lpel3pRFAzlow7u2TJMz4Q,4,3,0,0,The server had an extensive knowledge of wines...,2015-04-28 19:28:12,1,server extensive knowledge wine various menu s...
2,2,RWmT90rP7wYo57dG1cyvww,n3ahpmnEXsZYj3nbVMoUmg,Zja7ykq3icDzkN20XX6j1w,5,1,0,1,Fast Forward\n\nEvery trip from NYC to Pitt in...,2013-07-12 23:08:35,2,fast forward every trip nyc pitt includes visi...
3,3,0jduztItUhOh3FzKGjMxZQ,lVbPEaNUxFjwUydjentrYw,hr0vxeTzlHx6fTpSSmu2AA,5,1,0,0,By far my favorite Cleveland/Akron restaurant!...,2013-12-07 01:09:23,3,far favorite cleveland/akron restaurant great ...
4,4,sxfmzhMvZ8LlI1x4fhNjKg,SQKRGzDa7qd8qHb13Z2-Hg,F4kjKxXXyn3e-BYVb1mWcg,4,0,0,0,All food is good here I usually get the Italia...,2014-10-27 01:42:40,4,food good usually get italian sub pizza would ...
...,...,...,...,...,...,...,...,...,...,...,...,...
67164,67164,gcNRaJMDlSVs8YlqwpsRuw,_Rgn4SZT2wXyQnW9AD9Tog,nR3u-wQ99bop6IdGoU38KQ,1,1,0,0,I'm originally from the other side of Ohio and...,2019-09-21 00:24:03,67164,originally side ohio always really enjoyed jet...
67165,67165,yFxDOdiDb8HPaK1io81kiw,4r33dXcE1oYZxjONrhxTiA,yT1Ad3auuw1jAJEZHr7Yqw,5,1,1,1,"Yes, yes, YES!! We loved going to the place in...",2019-09-03 04:32:16,67165,yes yes yes loved going place shadyside ecstat...
67166,67166,Z_xmoLgIMB4As90t0a5eBQ,Iy6HkAzFcpeU7GF0e1FRSA,R13VkcyUIU3xG2fRFbBvBA,4,3,3,3,Went here for a lunch meeting amidst s typical...,2019-12-11 16:18:26,67166,went lunch meeting amidst typical suburban str...
67167,67167,CzYRFCnkbqOVb3WIMMrbfw,TP9Tz7gcAfK-BeT16qpZ2w,M-caugKzJ9qLOmCBfD9Nyg,1,1,0,0,My wife and I have eaten at La Brioche at leas...,2017-06-13 02:14:53,67167,wife eaten la brioche least dozen time food ne...


In [47]:
business_review=pizza_business["business_id"]

In [48]:
ID=[]
Add=[]
Keep=[]
Fix=[]
Avoid=[]
for i in business_review.tolist():
    ID.append(i)
    business_add=[]
    business_keep=[]
    business_fix=[]
    business_avoid=[]
    target_review=" ".join(pizza_business_review[pizza_business_review["business_id"]==i]["clean_review"].tolist())
    for i in Positive_Noun:
        if i in target_review:
            business_keep.append(i)
        else:
            business_add.append(i)
    for i in Negative_Noun:
        if i in target_review:
            business_fix.append(i)
        else:
            business_avoid.append(i)   
            
    
    Add.append(" ".join(business_add))
    Keep.append(" ".join(business_keep))
    Fix.append(" ".join(business_fix))
    Avoid.append(" ".join(business_avoid))
    

In [49]:
Noun_Suggestion=pd.DataFrame([ID,Add,Keep,Fix,Avoid]).transpose()

In [50]:
Noun_Suggestion=Noun_Suggestion.rename(columns={0: "business_id", 1: "Add", 2: "Keep", 3: "Fix", 4: "Avoid"})

In [51]:
Noun_Suggestion

,business_id,Add,Keep,Fix,Avoid
0,ZkzutF0P_u0C0yTulwaHkA,beer meatball wine dessert mozzarella parmesan...,pepperoni sausage pepper bacon gluten rib hone...,mushroom lettuce cracker,shrimp spaghetti caesar noodle broccoli season...
1,39lLJK_rrYY2NYomSsQdUA,pepperoni meatball wine sausage dessert bacon ...,beer pepper parmesan rib,cracker,mushroom shrimp spaghetti lettuce caesar noodl...
2,0y6alZmSLnPzmG5_kP5Quw,pepperoni meatball wine bacon mozzarella glute...,beer sausage pepper dessert parmesan,mushroom shrimp,spaghetti lettuce caesar noodle broccoli seaso...
3,_Kp1IPTi17wBywYOd30raA,beer pepperoni wine sausage pepper dessert bac...,meatball rib,,mushroom shrimp spaghetti lettuce caesar noodl...
4,crBw_Hq3gCkppirgjnIl4A,wine dessert bacon gluten parmesan lasagna gel...,beer pepperoni meatball sausage pepper mozzare...,mushroom spaghetti lettuce caesar,shrimp noodle broccoli seasoning ketchup patty...
...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,beer meatball wine parmesan lasagna gelato coc...,pepperoni sausage pepper dessert bacon mozzare...,cracker,mushroom shrimp spaghetti lettuce caesar noodl...
1556,ZZGs9idRiKRHue2RyXw9uA,beer pepperoni meatball wine sausage pepper mo...,dessert bacon rib,,mushroom shrimp spaghetti lettuce caesar noodl...
1557,v8GgrqLTs_NZZl2tkyvOgA,beer pepperoni meatball sausage pepper dessert...,wine gluten rib balsamic,lettuce,mushroom shrimp spaghetti caesar noodle brocco...
1558,qWcaGio9JsqrvgmQiIRtIQ,beer meatball bacon mozzarella gluten lasagna ...,pepperoni wine sausage pepper dessert parmesan...,mushroom caesar broccoli fryer,shrimp spaghetti lettuce noodle seasoning ketc...


Check. For example, the business_id ZkzutF0P_u0C0yTulwaHkA, its review is:

In [52]:
target_review=" ".join(pizza_business_review[pizza_business_review["business_id"]=="ZkzutF0P_u0C0yTulwaHkA"]["clean_review"].tolist())

In [53]:
"mushroom" in target_review

True

In [54]:
"beer" in target_review

False

In [55]:
"pepperoni" in target_review

True

In [56]:
"shrimp" in target_review

False

No problem. 

# Review suggestion

In review suggestion, we will calcualte the positive review percentage of a word by business. Our group once want to set a threshold that positive percentage larger we hold that business show positive attitude toward your XXX service or word. But in the end we want to give the number and let business itself to decide customers attitude toward their service and food. 

In [57]:
Review_results

,Unnamed: 0,Word,T_Test_P_Value,Wilcoxon_Test_P_Value,T_Sig,Wilcoxon_Sig,Mean_Difference
0,1,pizza,0.000000e+00,0.000000e+00,True,True,1.769981
1,2,food,0.000000e+00,0.000000e+00,True,True,1.928271
2,3,cheese,3.078739e-178,5.050809e-211,True,True,1.439742
3,4,sauce,2.290198e-202,6.429506e-242,True,True,1.475058
4,5,service,0.000000e+00,0.000000e+00,True,True,2.279542
...,...,...,...,...,...,...,...
157,158,broth,2.881508e-05,7.269379e-06,True,True,0.935678
158,159,pear,3.226965e-16,5.662321e-16,True,True,0.938006
159,160,condiment,2.958286e-04,3.959045e-04,True,True,1.171371
160,161,cod,1.385416e-10,9.828975e-10,True,True,1.783448


In [58]:
Review_Word=Review_results["Word"].tolist()

In [59]:
Col=["review_id","business_id","stars",'pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'fry',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'potato',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'gluten',
 'parmesan',
 'spinach',
 'beef',
 'egg',
 'brunch',
 'ice',
 'chef',
 'breadstick',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'gelato',
 'cocktail',
 'eggplant',
 'gnocchi',
 'butter',
 'ravioli',
 'taco',
 'pesto',
 'margherita',
 'chocolate',
 'coffee',
 'takeout',
 'management',
 'building',
 'tv',
 'ricotta',
 'soda',
 'rib',
 'calamari',
 'attitude',
 'crunchy',
 'arugula',
 'vegetable',
 'salt',
 'artichoke',
 'margarita',
 'seafood',
 'crab',
 'prosciutto',
 'caesar',
 'veal',
 'bean',
 'grease',
 'lemon',
 'honey',
 'spice',
 'noodle',
 'buffet',
 'worker',
 'bun',
 'doughy',
 'cooking',
 'diner',
 'environment',
 'tea',
 'broccoli',
 'feta',
 'ale',
 'ham',
 'napkin',
 'beverage',
 'recipe',
 'toast',
 'fee',
 'pickle',
 'fat',
 'apple',
 'pita',
 'cafe',
 'sugar',
 'cookie',
 'wing',
 'topping',
 'dressing',
 'olive',
 'serving',
 'seasoning',
 'hummus',
 'corn',
 'ketchup',
 'iceberg',
 'tuna',
 'cucumber',
 'fig',
 'juice',
 'diet',
 'jam',
 'leg',
 'broth',
 'pear',
 'condiment',
 'cod',
 'pea']

In [60]:
Word_Review_Matrix_New=Word_Review_Matrix[Col]
Word_Review_Matrix_New

,review_id,business_id,stars,pizza,food,cheese,sauce,service,salad,restaurant,...,fig,juice,diet,jam,leg,broth,pear,condiment,cod,pea
0,hfXVxrlD6fr1kXfz6sR13Q,J9f-9Prw2YVM-fiZqv2fmQ,2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,EDQpLLFiAxeSxWI5ipwzBQ,lpel3pRFAzlow7u2TJMz4Q,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RWmT90rP7wYo57dG1cyvww,Zja7ykq3icDzkN20XX6j1w,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0jduztItUhOh3FzKGjMxZQ,hr0vxeTzlHx6fTpSSmu2AA,5,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sxfmzhMvZ8LlI1x4fhNjKg,F4kjKxXXyn3e-BYVb1mWcg,4,1.0,1.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67164,gcNRaJMDlSVs8YlqwpsRuw,nR3u-wQ99bop6IdGoU38KQ,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67165,yFxDOdiDb8HPaK1io81kiw,yT1Ad3auuw1jAJEZHr7Yqw,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67166,Z_xmoLgIMB4As90t0a5eBQ,R13VkcyUIU3xG2fRFbBvBA,4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67167,CzYRFCnkbqOVb3WIMMrbfw,M-caugKzJ9qLOmCBfD9Nyg,1,NaN,1.0,NaN,NaN,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0


In [61]:
business_review=pizza_business["business_id"]

In [62]:
ID=[]
Word=[]
for i in business_review.tolist():
    ID.append(i)
    Word_Results=[]
    Target_Review=Word_Review_Matrix_New[Word_Review_Matrix_New["business_id"]==i]
    for j in Review_Word:
        results=" "
        Positive_Review_Num=Target_Review[j].tolist().count(1)
        Negative_Review_Num=Target_Review[j].tolist().count(-1)
        Neutral_Review_Num=Target_Review[j].tolist().count(0)
        Sum=Positive_Review_Num+Negative_Review_Num+Neutral_Review_Num
        # Only consider the sum of review larger than 5. 
        if Sum>5:
            results=j+" "+str(round(max(Positive_Review_Num*100/Sum,0)))+"% "+str(Sum)
            Word_Results.append(results)
                
    Word.append(Word_Results)
        
    

In [63]:
Word

[['pizza 82% 28',
  'food 80% 10',
  'sauce 92% 12',
  'service 70% 10',
  'crust 67% 6',
  'garlic 57% 7',
  'ice 79% 14'],
 ['pizza 78% 9'],
 ['pizza 71% 21',
  'sauce 86% 7',
  'crust 67% 6',
  'price 75% 8',
  'dough 33% 6',
  'ice 67% 9',
  'wing 50% 6',
  'topping 50% 6'],
 ['pizza 67% 9'],
 ['pizza 85% 55',
  'food 80% 15',
  'cheese 67% 12',
  'sauce 76% 17',
  'service 62% 16',
  'salad 78% 9',
  'crust 63% 19',
  'delivery 33% 6',
  'staff 88% 8',
  'bread 89% 9',
  'flavor 88% 8',
  'pie 50% 12',
  'pepperoni 50% 6',
  'meatball 67% 6',
  'sausage 75% 8',
  'garlic 86% 7',
  'meat 71% 7',
  'pepper 62% 8',
  'ice 80% 30',
  'spaghetti 70% 10'],
 [],
 ['pizza 85% 13',
  'food 59% 22',
  'cheese 43% 7',
  'sauce 71% 14',
  'service 72% 18',
  'salad 29% 14',
  'restaurant 71% 14',
  'menu 57% 7',
  'staff 71% 7',
  'price 67% 9',
  'bread 58% 12',
  'dinner 25% 8',
  'meal 50% 10',
  'pasta 79% 14',
  'flavor 62% 8',
  'meatball 56% 9',
  'sausage 57% 7',
  'meat 64% 11',
  'p

In [64]:
Review_Suggestion=pd.DataFrame([ID,Word]).transpose()

In [65]:
Review_Suggestion=Review_Suggestion.rename(columns={0: "business_id", 1: "Positive_Rate"})

In [66]:
Review_Suggestion

,business_id,Positive_Rate
0,ZkzutF0P_u0C0yTulwaHkA,"[pizza 82% 28, food 80% 10, sauce 92% 12, serv..."
1,39lLJK_rrYY2NYomSsQdUA,[pizza 78% 9]
2,0y6alZmSLnPzmG5_kP5Quw,"[pizza 71% 21, sauce 86% 7, crust 67% 6, price..."
3,_Kp1IPTi17wBywYOd30raA,[pizza 67% 9]
4,crBw_Hq3gCkppirgjnIl4A,"[pizza 85% 55, food 80% 15, cheese 67% 12, sau..."
...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,"[pizza 81% 52, food 64% 11, cheese 38% 8, sauc..."
1556,ZZGs9idRiKRHue2RyXw9uA,[pizza 44% 9]
1557,v8GgrqLTs_NZZl2tkyvOgA,[food 60% 10]
1558,qWcaGio9JsqrvgmQiIRtIQ,"[pizza 87% 38, sauce 57% 7, crust 82% 17, deli..."


Check, for business_id ZkzutF0P_u0C0yTulwaHkA, count review from pizza:

In [67]:
Target_Review=Word_Review_Matrix_New[Word_Review_Matrix_New["business_id"]=="ZkzutF0P_u0C0yTulwaHkA"]

In [68]:
Target_Review["pizza"].value_counts()

 1.0    23
 0.0     3
-1.0     2
Name: pizza, dtype: int64

No problem

In [69]:
Suggestion=pd.merge(Business_Suggestion,Noun_Suggestion,how='left', on='business_id')
Suggestion=pd.merge(Suggestion,Review_Suggestion,how='left', on='business_id')
Suggestion

,business_id,Alcohol,BikeParking,GoodForKids,NoiseLevel,RestaurantsGoodForGroups,RestaurantsReservations,WiFi,RestaurantsDelivery,BusinessParking,Add,Keep,Fix,Avoid,Positive_Rate
0,ZkzutF0P_u0C0yTulwaHkA,0,1,1,1,1,1,0,0,0,beer meatball wine dessert mozzarella parmesan...,pepperoni sausage pepper bacon gluten rib hone...,mushroom lettuce cracker,shrimp spaghetti caesar noodle broccoli season...,"[pizza 82% 28, food 80% 10, sauce 92% 12, serv..."
1,39lLJK_rrYY2NYomSsQdUA,0,0,1,1,0,0,0,0,0,pepperoni meatball wine sausage dessert bacon ...,beer pepper parmesan rib,cracker,mushroom shrimp spaghetti lettuce caesar noodl...,[pizza 78% 9]
2,0y6alZmSLnPzmG5_kP5Quw,0,1,1,1,1,0,0,0,0,pepperoni meatball wine bacon mozzarella glute...,beer sausage pepper dessert parmesan,mushroom shrimp,spaghetti lettuce caesar noodle broccoli seaso...,"[pizza 71% 21, sauce 86% 7, crust 67% 6, price..."
3,_Kp1IPTi17wBywYOd30raA,1,0,1,1,1,0,0,0,1,beer pepperoni wine sausage pepper dessert bac...,meatball rib,,mushroom shrimp spaghetti lettuce caesar noodl...,[pizza 67% 9]
4,crBw_Hq3gCkppirgjnIl4A,1,1,1,1,1,0,1,0,1,wine dessert bacon gluten parmesan lasagna gel...,beer pepperoni meatball sausage pepper mozzare...,mushroom spaghetti lettuce caesar,shrimp noodle broccoli seasoning ketchup patty...,"[pizza 85% 55, food 80% 15, cheese 67% 12, sau..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,0,0,1,0,1,0,0,0,0,beer meatball wine parmesan lasagna gelato coc...,pepperoni sausage pepper dessert bacon mozzare...,cracker,mushroom shrimp spaghetti lettuce caesar noodl...,"[pizza 81% 52, food 64% 11, cheese 38% 8, sauc..."
1556,ZZGs9idRiKRHue2RyXw9uA,0,0,1,1,1,0,0,0,0,beer pepperoni meatball wine sausage pepper mo...,dessert bacon rib,,mushroom shrimp spaghetti lettuce caesar noodl...,[pizza 44% 9]
1557,v8GgrqLTs_NZZl2tkyvOgA,0,0,1,0,1,0,1,1,0,beer pepperoni meatball sausage pepper dessert...,wine gluten rib balsamic,lettuce,mushroom shrimp spaghetti caesar noodle brocco...,[food 60% 10]
1558,qWcaGio9JsqrvgmQiIRtIQ,0,1,1,1,1,0,0,0,1,beer meatball bacon mozzarella gluten lasagna ...,pepperoni wine sausage pepper dessert parmesan...,mushroom caesar broccoli fryer,shrimp spaghetti lettuce noodle seasoning ketc...,"[pizza 87% 38, sauce 57% 7, crust 82% 17, deli..."


In [70]:
Suggestion.to_csv("data/Suggestion.csv")